In [ ]:
CALL algo.betweenness.stream(
'MATCH (n:CountryYear) WHERE n.year=2009 RETURN id(n) AS id',
'MATCH (n)-[r:RESIDE_IN]->(m) WHERE n.year=2016 and m.year=2016 RETURN id(n) AS source, id(m) AS target',
{graph: "cypher"})

YIELD nodeId, centrality

MATCH (CountryYear) WHERE id(CountryYear) = nodeId
RETURN CountryYear.countryearId AS countryyear,centrality
ORDER BY centrality DESC;